# Agentic RAG with langchain-oracledb

This notebook demonstrates the **langchain-oracledb** integration in the **Agentic RAG** application, showcasing how Oracle Database 26ai powers an intelligent multi-agent RAG system.

## What You'll Learn

1. **langchain-oracledb Components**
   - `OracleVS` - Vector store for similarity search
   - `OracleEmbeddings` - In-database embedding generation
   - `OracleTextSplitter` - Server-side text chunking

2. **Multi-Collection Vector Store**
   - PDF, Web, Repository, and General Knowledge collections
   - Metadata management and sanitization
   - Similarity search across collections

3. **Multi-Agent Chain of Thought (CoT)**
   - Planner, Researcher, Reasoner, Synthesizer agents
   - A2A protocol for agent communication
   - Distributed agent architecture

## Table of Contents

1. [Setup and Configuration](#1-setup-and-configuration)
2. [Oracle Database Connection](#2-oracle-database-connection)
3. [langchain-oracledb Core Components](#3-langchain-oracledb-core-components)
   - 3.1 OracleEmbeddings
   - 3.2 OracleVS (Vector Store)
   - 3.3 OracleTextSplitter
4. [Multi-Collection Vector Store](#4-multi-collection-vector-store)
5. [Document Processing Pipeline](#5-document-processing-pipeline)
6. [End-to-End langchain-oracledb Workflow](#6-end-to-end-langchain-oracledb-workflow)
7. [RAG Query Workflow](#7-rag-query-workflow)
8. [Multi-Agent Architecture](#8-multi-agent-architecture)
9. [A2A Protocol Integration](#9-a2a-protocol-integration)

---
## 1. Setup and Configuration

In [21]:
# Core imports
import sys
import os
from pathlib import Path
import json
import yaml
import time
import warnings
warnings.filterwarnings('ignore')

# Set up path to agentic_rag - handle both execution contexts:
# 1. Running from notebooks/ directory (manual Jupyter)
# 2. Running from agentic_rag/ directory (papermill with --cwd)

cwd = Path.cwd()
project_root = Path(__file__).parent.parent if '__file__' in dir() else cwd

# Detect if we're already in agentic_rag directory
if cwd.name == "agentic_rag" or (cwd / "src" / "OraDBVectorStore.py").exists():
    AGENTIC_RAG_PATH = cwd
elif (cwd.parent / "apps" / "agentic_rag").exists():
    AGENTIC_RAG_PATH = (cwd.parent / "apps" / "agentic_rag").resolve()
elif (cwd / "apps" / "agentic_rag").exists():
    AGENTIC_RAG_PATH = (cwd / "apps" / "agentic_rag").resolve()
else:
    # Fallback - search upward for project root
    search_path = cwd
    while search_path != search_path.parent:
        if (search_path / "apps" / "agentic_rag").exists():
            AGENTIC_RAG_PATH = (search_path / "apps" / "agentic_rag").resolve()
            break
        search_path = search_path.parent
    else:
        raise FileNotFoundError("Could not locate agentic_rag directory")

sys.path.insert(0, str(AGENTIC_RAG_PATH))
os.chdir(AGENTIC_RAG_PATH)

print(f"Working directory: {os.getcwd()}")
print(f"Agentic RAG path: {AGENTIC_RAG_PATH}")

Working directory: /home/ubuntu/git/oracle-ai-developer-hub/apps/agentic_rag
Agentic RAG path: /home/ubuntu/git/oracle-ai-developer-hub/apps/agentic_rag


In [22]:
# Check langchain-oracledb availability
try:
    from langchain_oracledb import OracleVS, OracleEmbeddings
    from langchain_oracledb.document_loaders.oracleai import OracleTextSplitter
    print("✅ langchain-oracledb installed")
    print("   Available components:")
    print("   • OracleVS - Vector store")
    print("   • OracleEmbeddings - Embedding generation")
    print("   • OracleTextSplitter - Text chunking")
except ImportError as e:
    print(f"❌ langchain-oracledb not available: {e}")
    print("   Install with: pip install langchain-oracledb")

✅ langchain-oracledb installed
   Available components:
   • OracleVS - Vector store
   • OracleEmbeddings - Embedding generation
   • OracleTextSplitter - Text chunking


In [23]:
# Load configuration
from src.db_utils import load_config

config = load_config()

print("📋 Configuration Loaded")
print("="*50)
print(f"Oracle Username: {config.get('ORACLE_DB_USERNAME', 'N/A')}")
print(f"Oracle DSN: {config.get('ORACLE_DB_DSN', 'N/A')[:80]}...")
print(f"Embedding Params: {config.get('ORACLE_EMBEDDINGS_PARAMS', {'provider': 'database', 'model': 'ALL_MINILM_L12_V2'})}")

📋 Configuration Loaded
Oracle Username: ADMIN
Oracle DSN: (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)...
Embedding Params: {'provider': 'database', 'model': 'ALL_MINILM_L12_V2'}


---
## 2. Oracle Database Connection

In [24]:
import oracledb
from src.db_utils import get_db_connection

# Establish connection
try:
    connection = get_db_connection(config)
    print("✅ Oracle Database connected!")
    
    # Get version info
    cursor = connection.cursor()
    cursor.execute("SELECT BANNER FROM V$VERSION WHERE ROWNUM = 1")
    version = cursor.fetchone()[0]
    print(f"   Version: {version}")
    cursor.close()
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    connection = None

Connecting (no wallet) to dsn (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.us-phoenix-1.oraclecloud.com))(connect_data=(service_name=g2f4dc3e5463897_mern_tpurgent.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes))) and user ADMIN
✅ Oracle Database connected!
   Version: Oracle AI Database 26ai Enterprise Edition Release 23.26.0.1.0 - for Oracle Cloud and Engineered Systems


In [25]:
# Check available ONNX models in the database
if connection:
    print("🔍 Checking ONNX Models in Database")
    print("="*50)
    
    cursor = connection.cursor()
    try:
        cursor.execute("SELECT MODEL_NAME, MINING_FUNCTION FROM USER_MINING_MODELS")
        models = cursor.fetchall()
        
        if models:
            print("Available models:")
            for name, func in models:
                print(f"   • {name} ({func})")
        else:
            print("   No ONNX models found in USER_MINING_MODELS")
            print("   Note: Models may be in a different schema")
    except Exception as e:
        print(f"   Could not query models: {e}")
    finally:
        cursor.close()

🔍 Checking ONNX Models in Database
Available models:
   • ALL_MINILM_L12_V2 (EMBEDDING)


---
## 3. langchain-oracledb Core Components

The agentic_rag application uses three key components from langchain-oracledb:

| Component | Purpose | Usage in Agentic RAG |
|-----------|---------|----------------------|
| **OracleEmbeddings** | Generate vector embeddings | Used by OraDBVectorStore for all collections |
| **OracleVS** | Store and search vectors | Multiple instances for PDF, Web, Repo, General |
| **OracleTextSplitter** | Chunk documents | Used by PDFProcessor for server-side splitting |

### 3.1 OracleEmbeddings

OracleEmbeddings generates vector embeddings using models loaded in Oracle Database. This eliminates external API calls and keeps data within the database.

In [26]:
from langchain_oracledb import OracleEmbeddings

if connection:
    print("🔢 OracleEmbeddings Demo")
    print("="*50)
    
    # Initialize with database provider (ONNX model)
    embed_params = config.get("ORACLE_EMBEDDINGS_PARAMS", 
                              {"provider": "database", "model": "ALL_MINILM_L12_V2"})
    
    if isinstance(embed_params, str):
        embed_params = json.loads(embed_params)
    
    print(f"Embedding params: {embed_params}")
    print("")
    
    try:
        embeddings = OracleEmbeddings(conn=connection, params=embed_params)
        print("✅ OracleEmbeddings initialized")
        
        # Generate embeddings for test texts
        test_texts = [
            "Oracle Database provides enterprise-grade AI capabilities.",
            "Vector search enables semantic similarity matching.",
            "Multi-agent systems coordinate to solve complex problems."
        ]
        
        print("")
        print("Generating embeddings for test texts...")
        start = time.time()
        vectors = embeddings.embed_documents(test_texts)
        elapsed = time.time() - start
        
        print(f"")
        print(f"Results:")
        print(f"   Texts embedded: {len(vectors)}")
        print(f"   Vector dimension: {len(vectors[0])}")
        print(f"   Time: {elapsed*1000:.2f} ms")
        print(f"   Sample (first 5 values): {vectors[0][:5]}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("   Note: Ensure ONNX model is loaded in Oracle Database")

🔢 OracleEmbeddings Demo
Embedding params: {'provider': 'database', 'model': 'ALL_MINILM_L12_V2'}

✅ OracleEmbeddings initialized

Generating embeddings for test texts...



Results:
   Texts embedded: 3
   Vector dimension: 384
   Time: 995.67 ms
   Sample (first 5 values): [0.0019562128, -0.0239340458, -0.0302166697, -0.0731686875, -0.0598886609]


### 3.2 OracleVS (Vector Store)

OracleVS provides vector storage and similarity search. In agentic_rag, it's used to create multiple collections for different document types.

In [27]:
from langchain_oracledb import OracleVS
from langchain_core.documents import Document

if connection and 'embeddings' in dir():
    print("📦 OracleVS Demo")
    print("="*50)
    
    # Create a test vector store
    test_store = OracleVS(
        client=connection,
        embedding_function=embeddings,
        table_name="NOTEBOOK_TEST_COLLECTION",
        distance_strategy="EUCLIDEAN_DISTANCE"
    )
    print("✅ OracleVS instance created")
    print(f"   Table: NOTEBOOK_TEST_COLLECTION")
    print(f"   Distance: EUCLIDEAN_DISTANCE")
    
    # Add test documents
    test_docs = [
        "Agentic RAG combines multi-agent reasoning with vector retrieval.",
        "The Planner agent breaks down complex queries into steps.",
        "The Researcher agent gathers information from vector stores.",
        "The Reasoner agent applies logical analysis to findings.",
        "The Synthesizer agent combines reasoning into final answers."
    ]
    
    test_metadatas = [
        {"source": "overview", "type": "concept"},
        {"source": "planner", "type": "agent"},
        {"source": "researcher", "type": "agent"},
        {"source": "reasoner", "type": "agent"},
        {"source": "synthesizer", "type": "agent"}
    ]
    
    print("")
    print(f"Adding {len(test_docs)} test documents...")
    test_store.add_texts(texts=test_docs, metadatas=test_metadatas)
    connection.commit()
    print("✅ Documents added")

📦 OracleVS Demo
✅ OracleVS instance created
   Table: NOTEBOOK_TEST_COLLECTION
   Distance: EUCLIDEAN_DISTANCE

Adding 5 test documents...
✅ Documents added


In [28]:
# Test similarity search
if 'test_store' in dir():
    print("🔍 Similarity Search Demo")
    print("="*50)
    
    query = "What agents are involved in the reasoning process?"
    print(f"Query: {query}")
    print("")
    
    results = test_store.similarity_search(query, k=3)
    
    print(f"Top {len(results)} results:")
    print("-"*40)
    for i, doc in enumerate(results):
        print(f"[{i+1}] {doc.page_content}")
        print(f"    Metadata: {doc.metadata}")
        print()

🔍 Similarity Search Demo
Query: What agents are involved in the reasoning process?

Top 3 results:
----------------------------------------
[1] The Reasoner agent applies logical analysis to findings.
    Metadata: {'source': 'reasoner', 'type': 'agent'}

[2] The Reasoner agent applies logical analysis to findings.
    Metadata: {'source': 'reasoner', 'type': 'agent'}

[3] The Synthesizer agent combines reasoning into final answers.
    Metadata: {'source': 'synthesizer', 'type': 'agent'}



### 3.3 OracleTextSplitter

OracleTextSplitter performs server-side text chunking with normalization. This is used by PDFProcessor to split documents before embedding.

In [29]:
from langchain_oracledb.document_loaders.oracleai import OracleTextSplitter

if connection:
    print("✂️ OracleTextSplitter Demo")
    print("="*50)
    
    # Initialize with normalize="all" (as used in agentic_rag)
    splitter_params = {"normalize": "all"}
    
    try:
        splitter = OracleTextSplitter(conn=connection, params=splitter_params)
        print("✅ OracleTextSplitter initialized")
        print(f"   Params: {splitter_params}")
        print("")
        
        # Test text (simulating a document)
        test_text = """
        # Multi-Agent RAG Architecture
        
        The agentic RAG system uses a Chain of Thought (CoT) approach with four 
        specialized agents working together to answer complex queries.
        
        ## Planner Agent
        The Planner agent receives the initial query and breaks it down into 
        manageable steps. It creates a strategic plan for addressing the question.
        
        ## Researcher Agent
        The Researcher agent searches the vector stores for relevant information.
        It gathers context from PDFs, web content, and repository code.
        
        ## Reasoner Agent
        The Reasoner agent applies logical analysis to the gathered information.
        It draws conclusions and identifies patterns in the data.
        
        ## Synthesizer Agent
        The Synthesizer agent combines all the reasoning steps into a coherent
        final answer that directly addresses the original query.
        """
        
        print(f"Input text: {len(test_text)} characters")
        
        # Split the text
        chunks = splitter.split_text(test_text)
        
        print(f"")
        print(f"Results:")
        print(f"   Chunks created: {len(chunks)}")
        print("")
        print("Chunks preview:")
        print("-"*40)
        for i, chunk in enumerate(chunks[:3]):
            preview = chunk[:100].replace('\n', ' ')
            print(f"[{i+1}] {preview}...")
        if len(chunks) > 3:
            print(f"... and {len(chunks)-3} more chunks")
            
    except Exception as e:
        print(f"❌ Error: {e}")

✂️ OracleTextSplitter Demo
✅ OracleTextSplitter initialized
   Params: {'normalize': 'all'}

Input text: 919 characters

Results:
   Chunks created: 2

Chunks preview:
----------------------------------------
[1] # Multi-Agent RAG Architecture  The agentic RAG system uses a Chain of Thought (CoT) approach with f...
[2] ## Reasoner Agent The Reasoner agent applies logical analysis to the gathered information. It draws ...


---
## 4. Multi-Collection Vector Store

The `OraDBVectorStore` class in agentic_rag manages multiple collections:

| Collection | Purpose | Table Name |
|------------|---------|------------|
| PDF | PDF documents | PDFCOLLECTION |
| Web | Web content | WEBCOLLECTION |
| Repository | Code/repos | REPOCOLLECTION |
| General | General knowledge | GENERALCOLLECTION |

In [30]:
from src.OraDBVectorStore import OraDBVectorStore

print("📦 OraDBVectorStore - Multi-Collection Demo")
print("="*50)

try:
    # Initialize the store (this creates OracleVS instances for each collection)
    ora_store = OraDBVectorStore()
    print("✅ OraDBVectorStore initialized")
    print("")
    
    # Show available collections
    print("Available collections:")
    for name, table in ora_store.collections.items():
        count = ora_store.get_collection_count(name)
        print(f"   • {name}: {count} documents (table: {table})")
        
except Exception as e:
    print(f"❌ Error: {e}")
    ora_store = None

📦 OraDBVectorStore - Multi-Collection Demo
Connecting (no wallet) to dsn (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.us-phoenix-1.oraclecloud.com))(connect_data=(service_name=g2f4dc3e5463897_mern_tpurgent.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes))) and user ADMIN
Oracle DB Connection successful!
✅ OraDBVectorStore initialized

Available collections:
   • PDFCOLLECTION: 92 documents (table: PDFCOLLECTION)
   • WEBCOLLECTION: 234 documents (table: WEBCOLLECTION)
   • REPOCOLLECTION: 2755 documents (table: REPOCOLLECTION)
   • GENERALCOLLECTION: 1 documents (table: GENERALCOLLECTION)


In [31]:
# Get detailed collection statistics
if ora_store:
    print("📊 Collection Statistics")
    print("="*50)
    
    for name in ora_store.collections:
        print(f"\n{name}:")
        print("-"*30)
        
        count = ora_store.get_collection_count(name)
        print(f"   Documents: {count}")
        
        if count > 0:
            dim = ora_store.get_embedding_dimension(name)
            print(f"   Embedding dimension: {dim}")
            
            # Get sample chunk
            sample = ora_store.get_latest_chunk(name)
            if sample:
                preview = str(sample.get('content', ''))[:80]
                print(f"   Sample: {preview}...")

📊 Collection Statistics

PDFCOLLECTION:
------------------------------
   Documents: 92
   Embedding dimension: 384
   Sample: Methods for physics-based character animation that use forward dynamic simulatio...

WEBCOLLECTION:
------------------------------
   Documents: 234
   Embedding dimension: 384
   Sample: In the seven seasons of giving out the Avco World Trophy, one of (or both) the W...

REPOCOLLECTION:
------------------------------
   Documents: 2755
   Embedding dimension: 384
   Sample: IN NO EVENT SHALL THE COPYRIGHT
// OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIREC...

GENERALCOLLECTION:
------------------------------
   Documents: 1
   Embedding dimension: 384
   Sample: # Test Document

This is a test document about machine learning.

Machine learni...


In [32]:
# Query different collections
if ora_store:
    print("🔍 Cross-Collection Query Demo")
    print("="*50)
    
    query = "How does the system process documents?"
    print(f"Query: {query}")
    print("")
    
    # Query each collection
    for collection in ["PDFCOLLECTION", "WEBCOLLECTION", "GENERALCOLLECTION"]:
        print(f"\n{collection}:")
        print("-"*30)
        
        try:
            if collection == "PDFCOLLECTION":
                results = ora_store.query_pdf_collection(query, n_results=2)
            elif collection == "WEBCOLLECTION":
                results = ora_store.query_web_collection(query, n_results=2)
            else:
                results = ora_store.query_general_collection(query, n_results=2)
            
            if results:
                for i, r in enumerate(results):
                    content = r.get('content', '')[:100]
                    source = r.get('metadata', {}).get('source', 'unknown')
                    print(f"   [{i+1}] {content}...")
                    print(f"       Source: {source}")
            else:
                print("   No results")
        except Exception as e:
            print(f"   Error: {e}")

🔍 Cross-Collection Query Demo
Query: How does the system process documents?


PDFCOLLECTION:
------------------------------
🔍 [OracleVS] Querying PDFCOLLECTION
🔍 [OracleVS] Retrieved 2 chunks from PDFCOLLECTION
   [1] Our creature model consists of a hierarchy of rigid bodies, which are actuated using an established ...
       Source: /tmp/gradio/20393179ac438e2cfb49d9c71edef977aeb0831599bc5ad856ca1b5cf3284654/2013-TOG-MuscleBasedBipeds.pdf
   [2] Our creature model consists of a hierarchy of rigid bodies, which are actuated using an established ...
       Source: /tmp/gradio/20393179ac438e2cfb49d9c71edef977aeb0831599bc5ad856ca1b5cf3284654/2013-TOG-MuscleBasedBipeds.pdf

WEBCOLLECTION:
------------------------------
🔍 [OracleVS] Querying WEBCOLLECTION
🔍 [OracleVS] Retrieved 2 chunks from WEBCOLLECTION
   [1] Although a constitution is a written document, there is also an unwritten constitution. The unwritte...
       Source: https://en.wikipedia.org/wiki/Politics
   [2] Political scien

---
## 5. Document Processing Pipeline

The `PDFProcessor` class demonstrates the full document processing pipeline:
1. **Load** - Docling extracts text from PDFs
2. **Split** - OracleTextSplitter chunks the text
3. **Store** - OraDBVectorStore adds chunks with embeddings

In [33]:
# Demonstrate the PDFProcessor pattern
print("📄 Document Processing Pipeline")
print("="*50)

# Show the processing flow
pipeline_steps = """
┌─────────────────────────────────────────────────────────────┐
│                  DOCUMENT PROCESSING PIPELINE               │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. LOAD (Docling)                                          │
│     ├── PDF → Markdown conversion                           │
│     └── Preserve document structure                         │
│                    ↓                                        │
│  2. SPLIT (OracleTextSplitter)                              │
│     ├── Server-side chunking                                │
│     ├── Text normalization                                  │
│     └── Configurable parameters                             │
│                    ↓                                        │
│  3. EMBED (OracleEmbeddings)                                │
│     ├── In-database ONNX model                              │
│     └── ALL_MINILM_L12_V2 (384 dims)                        │
│                    ↓                                        │
│  4. STORE (OracleVS)                                        │
│     ├── Vector + metadata storage                           │
│     └── Collection-based organization                       │
│                                                             │
└─────────────────────────────────────────────────────────────┘
"""
print(pipeline_steps)

📄 Document Processing Pipeline

┌─────────────────────────────────────────────────────────────┐
│                  DOCUMENT PROCESSING PIPELINE               │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. LOAD (Docling)                                          │
│     ├── PDF → Markdown conversion                           │
│     └── Preserve document structure                         │
│                    ↓                                        │
│  2. SPLIT (OracleTextSplitter)                              │
│     ├── Server-side chunking                                │
│     ├── Text normalization                                  │
│     └── Configurable parameters                             │
│                    ↓                                        │
│  3. EMBED (OracleEmbeddings)                                │
│     ├── In-database ONNX model                              │
│     └─

In [34]:
# Simulate the processing pipeline
if connection and ora_store:
    print("🔄 Simulating Document Processing")
    print("="*50)
    
    # Sample document content (simulating extracted PDF text)
    sample_document = """
    # Oracle AI Database 26ai Features
    
    Oracle Database 26ai introduces powerful AI capabilities directly within 
    the database engine, enabling developers to build intelligent applications
    without moving data to external systems.
    
    ## Vector Search
    Native support for vector similarity search with HNSW and IVF indexing.
    Enables semantic search across documents and unstructured data.
    
    ## In-Database ML
    Run machine learning models directly in the database using ONNX format.
    Supports embedding generation and inference without data movement.
    
    ## langchain-oracledb Integration
    Seamless integration with LangChain for building RAG applications.
    Components include OracleVS, OracleEmbeddings, and OracleTextSplitter.
    """
    
    print(f"Input: {len(sample_document)} characters")
    print("")
    
    # Step 1: Split using OracleTextSplitter
    print("Step 1: Splitting with OracleTextSplitter...")
    try:
        splitter = OracleTextSplitter(conn=connection, params={"normalize": "all"})
        chunks = splitter.split_text(sample_document)
        print(f"   ✅ Created {len(chunks)} chunks")
    except Exception as e:
        print(f"   ❌ Splitter error: {e}")
        chunks = [sample_document]  # Fallback
    
    # Step 2: Create chunk objects with metadata
    print("")
    print("Step 2: Creating chunk metadata...")
    import uuid
    document_id = str(uuid.uuid4())
    
    processed_chunks = []
    for i, chunk_text in enumerate(chunks):
        processed_chunks.append({
            "text": chunk_text,
            "metadata": {
                "source": "notebook_demo.pdf",
                "document_id": document_id,
                "chunk_index": i
            }
        })
    print(f"   ✅ Prepared {len(processed_chunks)} chunks with metadata")
    
    # Step 3: Add to vector store (optional - uncomment to actually store)
    print("")
    print("Step 3: Ready for vector store insertion")
    print(f"   Document ID: {document_id}")
    print(f"   Collection: PDFCOLLECTION")
    print("   (Skipping actual insertion for demo)")
    
    # To actually insert:
    # ora_store.add_pdf_chunks(processed_chunks, document_id)

🔄 Simulating Document Processing
Input: 768 characters

Step 1: Splitting with OracleTextSplitter...
   ✅ Created 2 chunks

Step 2: Creating chunk metadata...
   ✅ Prepared 2 chunks with metadata

Step 3: Ready for vector store insertion
   Document ID: f0321bb3-cfce-49c7-ab1a-19744e7d38d4
   Collection: PDFCOLLECTION
   (Skipping actual insertion for demo)


---
## 6. End-to-End langchain-oracledb Workflow

This section demonstrates the complete workflow used in the Agentic RAG application:
1. **Add a document** - Create a sample document
2. **Process with langchain-oracledb** - Split and embed using `OracleTextSplitter` and `OracleEmbeddings`
3. **Store in OracleVS** - Add to vector store
4. **Query with similarity search** - Use `OraDBVectorStore` to retrieve relevant content

This mirrors exactly how the production system processes and queries documents.

In [35]:
# Step 1: Create a sample document
print("📄 Step 1: Create Sample Document")
print("="*60)

# This simulates a document that would be uploaded to the system
sample_document_content = """
# Oracle AI Vector Search Guide

Oracle AI Vector Search enables semantic similarity search within Oracle Database.
It allows you to store, index, and query vector embeddings alongside your 
relational data without requiring external vector databases.

## Key Features

### In-Database Embeddings
Generate embeddings directly in the database using ONNX models. This eliminates
the need for external API calls and keeps data processing within your security
perimeter. Supported models include ALL_MINILM_L12_V2 for general-purpose embeddings.

### Vector Indexing
Oracle supports multiple indexing strategies:
- HNSW (Hierarchical Navigable Small World): Best for approximate nearest neighbor
- IVF (Inverted File): Good balance of speed and accuracy
- Flat: Exact search, best for small datasets

### Distance Metrics
Choose from multiple distance strategies:
- EUCLIDEAN_DISTANCE: Standard L2 distance
- COSINE_DISTANCE: Angle-based similarity
- DOT_PRODUCT: Inner product similarity

## Integration with LangChain

The langchain-oracledb package provides seamless integration:

```python
from langchain_oracledb import OracleVS, OracleEmbeddings
from langchain_oracledb.document_loaders.oracleai import OracleTextSplitter

# Initialize embeddings with in-database ONNX model
embeddings = OracleEmbeddings(conn=connection, params={"provider": "database", "model": "ALL_MINILM_L12_V2"})

# Create vector store
vector_store = OracleVS(client=connection, embedding_function=embeddings, table_name="MY_COLLECTION")

# Add documents and search
vector_store.add_texts(texts, metadatas)
results = vector_store.similarity_search("my query", k=5)
```

## Best Practices

1. **Chunking Strategy**: Use OracleTextSplitter for consistent server-side chunking
2. **Metadata Management**: Store source information, timestamps, and document IDs
3. **Index Selection**: Use HNSW for large datasets, Flat for small ones
4. **Batch Processing**: Add documents in batches for better performance
"""

print(f"Document title: Oracle AI Vector Search Guide")
print(f"Document size: {len(sample_document_content)} characters")
print(f"")
print("Preview (first 300 chars):")
print("-"*60)
print(sample_document_content[:300] + "...")

📄 Step 1: Create Sample Document
Document title: Oracle AI Vector Search Guide
Document size: 1978 characters

Preview (first 300 chars):
------------------------------------------------------------

# Oracle AI Vector Search Guide

Oracle AI Vector Search enables semantic similarity search within Oracle Database.
It allows you to store, index, and query vector embeddings alongside your 
relational data without requiring external vector databases.

## Key Features

### In-Database Embeddings
Ge...


In [36]:
# Step 2: Process document with langchain-oracledb
print("✂️ Step 2: Process with langchain-oracledb")
print("="*60)

from langchain_oracledb import OracleVS, OracleEmbeddings
from langchain_oracledb.document_loaders.oracleai import OracleTextSplitter
from src.db_utils import get_db_connection, load_config
import uuid

# Get fresh connection for this demo
demo_config = load_config()
demo_connection = get_db_connection(demo_config)
print("✅ Database connection established")

# Initialize OracleTextSplitter (same as used in PDFProcessor)
splitter_params = {"normalize": "all"}
demo_splitter = OracleTextSplitter(conn=demo_connection, params=splitter_params)
print(f"✅ OracleTextSplitter initialized with params: {splitter_params}")

# Split the document into chunks
chunks = demo_splitter.split_text(sample_document_content)
print(f"")
print(f"📊 Chunking Results:")
print(f"   Input: {len(sample_document_content)} characters")
print(f"   Output: {len(chunks)} chunks")
print(f"")
print("Chunk details:")
print("-"*60)
for i, chunk in enumerate(chunks):
    print(f"   Chunk {i+1}: {len(chunk)} chars")
    preview = chunk[:80].replace('\n', ' ').strip()
    print(f"   Preview: {preview}...")

✂️ Step 2: Process with langchain-oracledb
Connecting (no wallet) to dsn (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.us-phoenix-1.oraclecloud.com))(connect_data=(service_name=g2f4dc3e5463897_mern_tpurgent.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes))) and user ADMIN


✅ Database connection established
✅ OracleTextSplitter initialized with params: {'normalize': 'all'}

📊 Chunking Results:
   Input: 1978 characters
   Output: 5 chunks

Chunk details:
------------------------------------------------------------
   Chunk 1: 540 chars
   Preview: # Oracle AI Vector Search Guide  Oracle AI Vector Search enables semantic simila...
   Chunk 2: 523 chars
   Preview: ### Vector Indexing Oracle supports multiple indexing strategies:-HNSW (Hierarch...
   Chunk 3: 431 chars
   Preview: ```python from langchain_oracledb import OracleVS, OracleEmbeddings from langcha...
   Chunk 4: 148 chars
   Preview: # Add documents and search vector_store.add_texts(texts, metadatas) results = ve...
   Chunk 5: 313 chars
   Preview: 1. **Chunking Strategy**: Use OracleTextSplitter for consistent server-side chun...


In [37]:
# Step 3: Store in OracleVS vector store
print("📦 Step 3: Store in OracleVS")
print("="*60)

# Initialize OracleEmbeddings (same pattern as OraDBVectorStore)
embed_params = demo_config.get("ORACLE_EMBEDDINGS_PARAMS", 
                               {"provider": "database", "model": "ALL_MINILM_L12_V2"})
if isinstance(embed_params, str):
    import json
    embed_params = json.loads(embed_params)

demo_embeddings = OracleEmbeddings(conn=demo_connection, params=embed_params)
print(f"✅ OracleEmbeddings initialized")
print(f"   Provider: {embed_params.get('provider')}")
print(f"   Model: {embed_params.get('model')}")

# Create a dedicated vector store for this demo
DEMO_TABLE = "DEMO_WORKFLOW_COLLECTION"

demo_vector_store = OracleVS(
    client=demo_connection,
    embedding_function=demo_embeddings,
    table_name=DEMO_TABLE,
    distance_strategy="EUCLIDEAN_DISTANCE"
)
print(f"")
print(f"✅ OracleVS vector store created")
print(f"   Table: {DEMO_TABLE}")
print(f"   Distance strategy: EUCLIDEAN_DISTANCE")

# Prepare chunks with metadata (same pattern as OraDBVectorStore.add_pdf_chunks)
document_id = str(uuid.uuid4())
document_source = "oracle_vector_search_guide.pdf"

texts = chunks
metadatas = [
    {
        "source": document_source,
        "document_id": document_id,
        "chunk_index": i,
        "total_chunks": len(chunks),
        "processed_by": "langchain-oracledb"
    }
    for i in range(len(chunks))
]

print(f"")
print(f"📝 Adding {len(texts)} chunks to vector store...")
start_time = time.time()

# Add texts with metadata (this generates embeddings and stores them)
demo_vector_store.add_texts(texts=texts, metadatas=metadatas)
demo_connection.commit()

elapsed = time.time() - start_time
print(f"✅ Chunks added successfully!")
print(f"   Document ID: {document_id}")
print(f"   Time elapsed: {elapsed*1000:.2f} ms")

📦 Step 3: Store in OracleVS
✅ OracleEmbeddings initialized
   Provider: database
   Model: ALL_MINILM_L12_V2

✅ OracleVS vector store created
   Table: DEMO_WORKFLOW_COLLECTION
   Distance strategy: EUCLIDEAN_DISTANCE

📝 Adding 5 chunks to vector store...
✅ Chunks added successfully!
   Document ID: 33ea4156-f087-4800-8885-f90d473afffc
   Time elapsed: 481.48 ms


In [38]:
# Step 4: Query with similarity search (using OraDBVectorStore pattern)
print("🔍 Step 4: Query with Similarity Search")
print("="*60)

# Method 1: Direct OracleVS similarity search
print("Method 1: Direct OracleVS.similarity_search()")
print("-"*60)

query = "What embedding models are supported for vector search?"
print(f"Query: {query}")
print("")

# Perform similarity search
results = demo_vector_store.similarity_search(query, k=2)

print(f"Retrieved {len(results)} results:")
print("")
for i, doc in enumerate(results):
    print(f"Result [{i+1}]:")
    print(f"   Content: {doc.page_content[:150]}...")
    print(f"   Metadata: {doc.metadata}")
    print()

🔍 Step 4: Query with Similarity Search
Method 1: Direct OracleVS.similarity_search()
------------------------------------------------------------
Query: What embedding models are supported for vector search?

Retrieved 2 results:

Result [1]:
   Content: # Oracle AI Vector Search Guide

Oracle AI Vector Search enables semantic similarity search within Oracle Database.
It allows you to store, index, and...
   Metadata: {'source': 'oracle_vector_search_guide.pdf', 'document_id': '33ea4156-f087-4800-8885-f90d473afffc', 'chunk_index': Decimal('0'), 'total_chunks': Decimal('5'), 'processed_by': 'langchain-oracledb'}

Result [2]:
   Content: # Oracle AI Vector Search Guide

Oracle AI Vector Search enables semantic similarity search within Oracle Database.
It allows you to store, index, and...
   Metadata: {'source': 'oracle_vector_search_guide.pdf', 'document_id': '321ff555-13f3-42bc-a3ed-db207e7c409d', 'chunk_index': Decimal('0'), 'total_chunks': Decimal('5'), 'processed_by': 'langchain-ora

In [39]:
# Method 2: Query with similarity scores
print("Method 2: OracleVS.similarity_search_with_score()")
print("-"*60)

query2 = "How do I integrate Oracle vector search with LangChain?"
print(f"Query: {query2}")
print("")

# Similarity search with scores (used by OraDBVectorStore._query_collection)
try:
    results_with_scores = demo_vector_store.similarity_search_with_score(query2, k=2)
    
    print(f"Retrieved {len(results_with_scores)} results with scores:")
    print("")
    for i, (doc, distance) in enumerate(results_with_scores):
        # Convert distance to similarity (same formula as OraDBVectorStore)
        similarity = 1 / (1 + distance) if distance >= 0 else 0
        
        print(f"Result [{i+1}]:")
        print(f"   Distance: {distance:.4f}")
        print(f"   Similarity: {similarity:.4f}")
        print(f"   Content: {doc.page_content[:120]}...")
        print()
except Exception as e:
    print(f"Note: similarity_search_with_score error: {e}")
    print("Falling back to similarity_search (same as OraDBVectorStore)")
    results = demo_vector_store.similarity_search(query2, k=2)
    for i, doc in enumerate(results):
        print(f"Result [{i+1}]: {doc.page_content[:120]}...")

Method 2: OracleVS.similarity_search_with_score()
------------------------------------------------------------
Query: How do I integrate Oracle vector search with LangChain?

Retrieved 2 results with scores:

Result [1]:
   Distance: 0.7999
   Similarity: 0.5556
   Content: # Oracle AI Vector Search Guide

Oracle AI Vector Search enables semantic similarity search within Oracle Database.
It a...

Result [2]:
   Distance: 0.7999
   Similarity: 0.5556
   Content: # Oracle AI Vector Search Guide

Oracle AI Vector Search enables semantic similarity search within Oracle Database.
It a...



In [40]:
# Method 3: Using OraDBVectorStore (production pattern)
print("Method 3: Using OraDBVectorStore (Production Pattern)")
print("-"*60)
print("")
print("This is how the Agentic RAG application performs queries.")
print("OraDBVectorStore wraps OracleVS with collection management and")
print("metadata handling.")
print("")

from src.OraDBVectorStore import OraDBVectorStore

# Create OraDBVectorStore instance (same as LocalRAGAgent uses)
production_store = OraDBVectorStore()

# Add our demo chunks to the GENERAL collection (for testing)
print("Adding demo document to GENERALCOLLECTION...")
demo_chunks = [
    {
        "text": chunk,
        "metadata": {
            "source": document_source,
            "document_id": document_id,
            "chunk_index": i
        }
    }
    for i, chunk in enumerate(chunks)
]

# Use the production method to add chunks
production_store.add_general_knowledge(demo_chunks, source_id=document_id)
production_store.connection.commit()
print(f"✅ Added {len(demo_chunks)} chunks to GENERALCOLLECTION")
print("")

# Query using the production method
query3 = "What distance metrics can I use for vector similarity?"
print(f"Query: {query3}")
print("")

results = production_store.query_general_collection(query3, n_results=3)

print(f"Retrieved {len(results)} results:")
for i, r in enumerate(results):
    print(f"")
    print(f"Result [{i+1}]:")
    print(f"   Content: {r['content'][:120]}...")
    print(f"   Source: {r['metadata'].get('source', 'unknown')}")
    if 'score' in r:
        print(f"   Similarity: {r['score']:.4f}")

Method 3: Using OraDBVectorStore (Production Pattern)
------------------------------------------------------------

This is how the Agentic RAG application performs queries.
OraDBVectorStore wraps OracleVS with collection management and
metadata handling.

Connecting (no wallet) to dsn (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.us-phoenix-1.oraclecloud.com))(connect_data=(service_name=g2f4dc3e5463897_mern_tpurgent.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes))) and user ADMIN
Oracle DB Connection successful!
Adding demo document to GENERALCOLLECTION...
🔄 [OraDB] Inserting 5 chunks into GENERALCOLLECTION...
✅ [OraDB] Successfully inserted 5 chunks.
✅ Added 5 chunks to GENERALCOLLECTION

Query: What distance metrics can I use for vector similarity?

🔍 [OracleVS] Querying GENERALCOLLECTION
🔍 [OracleVS] Retrieved 3 chunks from GENERALCOLLECTION
Retrieved 3 results:

Result [1]:
   Content: ### Vector Indexing
Oracle supports multipl

In [41]:
# Method 4: Using LocalRAGAgent (Full Agentic RAG Pattern)
print("Method 4: Using LocalRAGAgent (Full Agentic RAG)")
print("-"*60)
print("")
print("LocalRAGAgent combines vector retrieval with LLM generation.")
print("This is the same agent used in the production Gradio UI.")
print("")

from src.local_rag_agent import LocalRAGAgent

# Initialize LocalRAGAgent with General Knowledge collection
# (where we just added our demo document)
rag_agent = LocalRAGAgent(
    vector_store=production_store,
    collection="General Knowledge"
)
print(f"✅ LocalRAGAgent initialized")
print(f"   Collection: General Knowledge")
print(f"   Vector Store: OraDBVectorStore")
print("")

# Perform a RAG query (retrieval + generation)
query4 = "Explain the vector indexing options available in Oracle"
print(f"Query: {query4}")
print("")

# Note: This requires Ollama to be running with a model
print("⚠️  Note: Full RAG response requires Ollama LLM")
print("   Demonstrating context retrieval step only...")
print("")

# Directly get context (what the agent does internally)
context_results = production_store.query_general_collection(query4, n_results=3)

print(f"📚 Retrieved Context ({len(context_results)} chunks):")
print("-"*60)
for i, r in enumerate(context_results):
    print(f"")
    print(f"[{i+1}] {r['content'][:200]}...")
    
print("")
print("💡 In production, LocalRAGAgent.generate_response() would:")
print("   1. Retrieve this context using OraDBVectorStore")
print("   2. Format a prompt with the context + query")
print("   3. Send to Ollama LLM for response generation")
print("   4. Return the final answer to the user")

Method 4: Using LocalRAGAgent (Full Agentic RAG)
------------------------------------------------------------

LocalRAGAgent combines vector retrieval with LLM generation.
This is the same agent used in the production Gradio UI.

LocalRAGAgent init - model_name: None
Using default model: gemma3:270m
Model Name after assignment: gemma3:270m

Loading Ollama model...
Model: gemma3:270m
Note: Make sure Ollama is running on your system.
Available Ollama models: mistral-small3.2:latest, qwen3:latest, glm-4.7-flash:latest, glm4:9b-chat-q4_K_M, codegemma:latest, codellama:latest, deepseek-coder:6.7b, gpt-oss:20b, gemma3:270m, gemma3:latest, gemma3:1b-it-qat, gemma3:4b-it-qat, mistral:latest, smollm2:135m, qwen3:0.6b, deepseek-r1:1.5b, llama3.2:3b, mistral:7b, phi3:3.8b, nomic-embed-text:latest, phi3:latest, mario:latest, llama3-backup:latest, mattw/pygmalion:latest, llama3.2:latest, qwq:latest, phi4:latest, llama2:latest, llama2:7b, qwen2:latest, deepseek-r1:latest, llama3:latest
Using Ollama 

In [42]:
# Cleanup: Remove demo data
print("🧹 Cleaning up demo data...")
print("="*60)

# Drop the demo table
try:
    cursor = demo_connection.cursor()
    cursor.execute(f"DROP TABLE {DEMO_TABLE} PURGE")
    demo_connection.commit()
    cursor.close()
    print(f"✅ Dropped demo table: {DEMO_TABLE}")
except Exception as e:
    print(f"   Note: Could not drop {DEMO_TABLE}: {e}")

# Remove demo chunks from GENERALCOLLECTION
try:
    cursor = production_store.connection.cursor()
    cursor.execute("""
        DELETE FROM GENERALCOLLECTION 
        WHERE JSON_VALUE(metadata, '$.document_id') = :doc_id
    """, {"doc_id": document_id})
    deleted_count = cursor.rowcount
    production_store.connection.commit()
    cursor.close()
    print(f"✅ Removed {deleted_count} demo chunks from GENERALCOLLECTION")
except Exception as e:
    print(f"   Note: Could not clean GENERALCOLLECTION: {e}")

# Close demo connection
demo_connection.close()
print(f"✅ Demo connection closed")
print("")
print("📋 Workflow Summary:")
print("-"*60)
print("   1. Created sample document (Oracle Vector Search Guide)")
print("   2. Split with OracleTextSplitter → chunks")
print("   3. Stored with OracleVS → embeddings + metadata")
print("   4. Queried with similarity_search → relevant context")
print("   5. Used OraDBVectorStore → production pattern")
print("   6. Used LocalRAGAgent → full agentic RAG workflow")

🧹 Cleaning up demo data...
✅ Dropped demo table: DEMO_WORKFLOW_COLLECTION
✅ Removed 5 demo chunks from GENERALCOLLECTION
✅ Demo connection closed

📋 Workflow Summary:
------------------------------------------------------------
   1. Created sample document (Oracle Vector Search Guide)
   2. Split with OracleTextSplitter → chunks
   3. Stored with OracleVS → embeddings + metadata
   4. Queried with similarity_search → relevant context
   5. Used OraDBVectorStore → production pattern
   6. Used LocalRAGAgent → full agentic RAG workflow


---
## 7. RAG Query Workflow

The LocalRAGAgent handles the query workflow:
1. Analyze query type
2. Retrieve relevant context from vector stores
3. Generate response using Ollama

In [43]:
# Demonstrate RAG query workflow
print("🔍 RAG Query Workflow")
print("="*50)

workflow_diagram = """
┌─────────────────────────────────────────────────────────────┐
│                    RAG QUERY WORKFLOW                       │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   User Query                                                │
│        ↓                                                    │
│   ┌─────────────────────────────────────┐                   │
│   │  1. Query Analysis (LocalRAGAgent)  │                   │
│   │     • Determine query type          │                   │
│   │     • Select target collection      │                   │
│   └─────────────────────────────────────┘                   │
│        ↓                                                    │
│   ┌─────────────────────────────────────┐                   │
│   │  2. Context Retrieval (OracleVS)    │                   │
│   │     • Embed query                   │                   │
│   │     • Similarity search             │                   │
│   │     • Return top-k chunks           │                   │
│   └─────────────────────────────────────┘                   │
│        ↓                                                    │
│   ┌─────────────────────────────────────┐                   │
│   │  3. Response Generation (Ollama)    │                   │
│   │     • Assemble context + query      │                   │
│   │     • Generate with LLM             │                   │
│   │     • Return answer                 │                   │
│   └─────────────────────────────────────┘                   │
│        ↓                                                    │
│   Response to User                                          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
"""
print(workflow_diagram)

🔍 RAG Query Workflow

┌─────────────────────────────────────────────────────────────┐
│                    RAG QUERY WORKFLOW                       │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   User Query                                                │
│        ↓                                                    │
│   ┌─────────────────────────────────────┐                   │
│   │  1. Query Analysis (LocalRAGAgent)  │                   │
│   │     • Determine query type          │                   │
│   │     • Select target collection      │                   │
│   └─────────────────────────────────────┘                   │
│        ↓                                                    │
│   ┌─────────────────────────────────────┐                   │
│   │  2. Context Retrieval (OracleVS)    │                   │
│   │     • Embed query                   │                   │
│   │     • Simila

In [44]:
# Demonstrate context retrieval (Step 2)
if ora_store:
    print("📚 Context Retrieval Demo")
    print("="*50)
    
    test_queries = [
        "What is vector search?",
        "How do agents communicate?",
        "Explain the embedding process"
    ]
    
    for query in test_queries:
        print(f"\nQuery: {query}")
        print("-"*40)
        
        # Try PDF collection first (mimicking LocalRAGAgent behavior)
        results = ora_store.query_pdf_collection(query, n_results=2)
        
        if results:
            print(f"Retrieved {len(results)} chunks:")
            for i, r in enumerate(results):
                content = r.get('content', '')[:80]
                print(f"   [{i+1}] {content}...")
        else:
            print("   No relevant context found")
            print("   (Would fall back to LLM knowledge)")

📚 Context Retrieval Demo

Query: What is vector search?
----------------------------------------
🔍 [OracleVS] Querying PDFCOLLECTION
🔍 [OracleVS] Retrieved 2 chunks from PDFCOLLECTION
Retrieved 2 chunks:
   [1] In our approach, we attempt to find efficient muscle routings through optimizati...
   [2] In our approach, we attempt to find efficient muscle routings through optimizati...

Query: How do agents communicate?
----------------------------------------
🔍 [OracleVS] Querying PDFCOLLECTION
🔍 [OracleVS] Retrieved 2 chunks from PDFCOLLECTION
Retrieved 2 chunks:
   [1] Our creature model consists of a hierarchy of rigid bodies, which are actuated u...
   [2] Our creature model consists of a hierarchy of rigid bodies, which are actuated u...

Query: Explain the embedding process
----------------------------------------
🔍 [OracleVS] Querying PDFCOLLECTION
🔍 [OracleVS] Retrieved 2 chunks from PDFCOLLECTION
Retrieved 2 chunks:
   [1] parameters. The first is the result of activation dynami

---
## 8. Multi-Agent Architecture

Agentic RAG uses a Chain of Thought (CoT) approach with four specialized agents:

| Agent | Role | Task |
|-------|------|------|
| **Planner** | Strategic planning | Break queries into 3-4 steps |
| **Researcher** | Information gathering | Search vector stores |
| **Reasoner** | Logical analysis | Draw conclusions |
| **Synthesizer** | Response generation | Combine into final answer |

In [45]:
from src.specialized_agent_cards import get_all_specialized_agent_cards

print("🤖 Multi-Agent Architecture")
print("="*50)

# Get all agent cards
agent_cards = get_all_specialized_agent_cards()

print(f"Total agents: {len(agent_cards)}")
print("")

# Show primary agents (v1)
for agent_id in ["planner_agent_v1", "researcher_agent_v1", "reasoner_agent_v1", "synthesizer_agent_v1"]:
    card = agent_cards.get(agent_id, {})
    print(f"📌 {card.get('name', agent_id)}")
    print(f"   ID: {card.get('agent_id')}")
    print(f"   Role: {card.get('metadata', {}).get('role', 'N/A')}")
    print(f"   Expertise: {', '.join(card.get('metadata', {}).get('expertise', []))}")
    print()

🤖 Multi-Agent Architecture
Total agents: 8

📌 Strategic Planner Agent
   ID: planner_agent_v1
   Role: Strategic Planner
   Expertise: problem_decomposition, strategic_planning, task_breakdown

📌 Information Researcher Agent
   ID: researcher_agent_v1
   Role: Information Gatherer
   Expertise: information_retrieval, knowledge_extraction, data_analysis

📌 Logic and Reasoning Agent
   ID: reasoner_agent_v1
   Role: Logic and Analysis
   Expertise: logical_reasoning, critical_thinking, pattern_recognition

📌 Information Synthesis Agent
   ID: synthesizer_agent_v1
   Role: Information Synthesizer
   Expertise: information_synthesis, summarization, coherent_writing



In [46]:
# Visualize Chain of Thought workflow
print("🔗 Chain of Thought (CoT) Workflow")
print("="*50)

cot_diagram = """
┌─────────────────────────────────────────────────────────────────┐
│                 CHAIN OF THOUGHT (CoT) WORKFLOW                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query: "Explain how the RAG system processes PDFs"      │
│                              ↓                                  │
│   ┌───────────────────────────────────────────────────────┐     │
│   │  PLANNER AGENT                                        │     │
│   │  "Let me break this down into steps:"                 │     │
│   │   Step 1: Understand PDF ingestion                    │     │
│   │   Step 2: Explain text extraction                     │     │
│   │   Step 3: Describe chunking process                   │     │
│   │   Step 4: Detail embedding storage                    │     │
│   └───────────────────────────────────────────────────────┘     │
│                              ↓                                  │
│   ┌───────────────────────────────────────────────────────┐     │
│   │  RESEARCHER AGENT (for each step)                     │     │
│   │  • Searches PDFCOLLECTION                             │     │
│   │  • Gathers relevant context                           │     │
│   │  • Extracts key findings                              │     │
│   └───────────────────────────────────────────────────────┘     │
│                              ↓                                  │
│   ┌───────────────────────────────────────────────────────┐     │
│   │  REASONER AGENT (for each step)                       │     │
│   │  • Analyzes findings                                  │     │
│   │  • Draws logical conclusions                          │     │
│   │  • Identifies patterns                                │     │
│   └───────────────────────────────────────────────────────┘     │
│                              ↓                                  │
│   ┌───────────────────────────────────────────────────────┐     │
│   │  SYNTHESIZER AGENT                                    │     │
│   │  • Combines all reasoning steps                       │     │
│   │  • Produces coherent final answer                     │     │
│   │  • Addresses original query                           │     │
│   └───────────────────────────────────────────────────────┘     │
│                              ↓                                  │
│   Final Answer: "The RAG system processes PDFs through..."      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
"""
print(cot_diagram)

🔗 Chain of Thought (CoT) Workflow

┌─────────────────────────────────────────────────────────────────┐
│                 CHAIN OF THOUGHT (CoT) WORKFLOW                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query: "Explain how the RAG system processes PDFs"      │
│                              ↓                                  │
│   ┌───────────────────────────────────────────────────────┐     │
│   │  PLANNER AGENT                                        │     │
│   │  "Let me break this down into steps:"                 │     │
│   │   Step 1: Understand PDF ingestion                    │     │
│   │   Step 2: Explain text extraction                     │     │
│   │   Step 3: Describe chunking process                   │     │
│   │   Step 4: Detail embedding storage                    │     │
│   └───────────────────────────────────────────────────────┘     │
│             

---
## 9. A2A Protocol Integration

The Agent-to-Agent (A2A) protocol enables distributed agent communication via JSON-RPC 2.0.

In [47]:
from src.a2a_models import AgentCard, AgentCapability, AgentEndpoint

print("🔌 A2A Protocol Demo")
print("="*50)

# Show A2A request format
a2a_request_example = {
    "jsonrpc": "2.0",
    "method": "document.query",
    "params": {
        "query": "How does vector search work?",
        "collection": "PDF",
        "use_cot": True
    },
    "id": "1"
}

print("A2A Request Format (JSON-RPC 2.0):")
print("-"*40)
print(json.dumps(a2a_request_example, indent=2))

print("")
print("Available A2A Methods:")
print("-"*40)
methods = [
    ("document.query", "Query with context retrieval"),
    ("document.upload", "Process and store documents"),
    ("agent.query", "Query specialized CoT agents"),
    ("agent.discover", "Find agents with capabilities"),
    ("agent.register", "Register new agents"),
    ("task.create", "Create long-running tasks"),
    ("task.status", "Check task status"),
    ("health.check", "System health")
]

for method, desc in methods:
    print(f"   • {method}: {desc}")

🔌 A2A Protocol Demo
A2A Request Format (JSON-RPC 2.0):
----------------------------------------
{
  "jsonrpc": "2.0",
  "method": "document.query",
  "params": {
    "query": "How does vector search work?",
    "collection": "PDF",
    "use_cot": true
  },
  "id": "1"
}

Available A2A Methods:
----------------------------------------
   • document.query: Query with context retrieval
   • document.upload: Process and store documents
   • agent.query: Query specialized CoT agents
   • agent.discover: Find agents with capabilities
   • agent.register: Register new agents
   • task.create: Create long-running tasks
   • task.status: Check task status
   • health.check: System health


In [48]:
# Show distributed architecture
print("🌐 Distributed Agent Architecture")
print("="*50)

distributed_diagram = """
┌─────────────────────────────────────────────────────────────────┐
│                 DISTRIBUTED AGENT DEPLOYMENT                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌─────────────────┐           ┌─────────────────┐             │
│   │  Client App     │    A2A    │  Main Server    │             │
│   │  (Gradio/API)   │ ◄──────► │  (Port 8000)    │             │
│   └─────────────────┘           └─────────────────┘             │
│                                        │                        │
│                    ┌───────────────────┼───────────────────┐    │
│                    │                   │                   │    │
│                    ▼                   ▼                   ▼    │
│   ┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐
│   │  Planner Agent  │   │ Researcher Agent│   │  Reasoner Agent │
│   │  (Server 1)     │   │  (Server 2)     │   │  (Server 3)     │
│   └─────────────────┘   └─────────────────┘   └─────────────────┘
│            │                    │                    │          │
│            └────────────────────┼────────────────────┘          │
│                                 ▼                               │
│                    ┌─────────────────────┐                      │
│                    │  Oracle Database    │                      │
│                    │  (Vector Store)     │                      │
│                    └─────────────────────┘                      │
│                                                                 │
│   Configuration (config.yaml):                                  │
│   AGENT_ENDPOINTS:                                              │
│     planner_url: http://server1:8000                            │
│     researcher_url: http://server2:8000                         │
│     reasoner_url: http://server3:8000                           │
│     synthesizer_url: http://server4:8000                        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
"""
print(distributed_diagram)

🌐 Distributed Agent Architecture

┌─────────────────────────────────────────────────────────────────┐
│                 DISTRIBUTED AGENT DEPLOYMENT                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌─────────────────┐           ┌─────────────────┐             │
│   │  Client App     │    A2A    │  Main Server    │             │
│   │  (Gradio/API)   │ ◄──────► │  (Port 8000)    │             │
│   └─────────────────┘           └─────────────────┘             │
│                                        │                        │
│                    ┌───────────────────┼───────────────────┐    │
│                    │                   │                   │    │
│                    ▼                   ▼                   ▼    │
│   ┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐
│   │  Planner Agent  │   │ Researcher Agent│   │  Reasoner Agent │
│   │  (Server 

---
## Cleanup

In [49]:
# Clean up test data
if 'test_store' in dir():
    print("🧹 Cleanup")
    print("="*50)
    
    try:
        cursor = connection.cursor()
        cursor.execute("DROP TABLE NOTEBOOK_TEST_COLLECTION PURGE")
        connection.commit()
        print("✅ Dropped test table NOTEBOOK_TEST_COLLECTION")
    except Exception as e:
        print(f"   Note: {e}")

# Close connections
if 'connection' in dir() and connection:
    connection.close()
    print("✅ Database connection closed")
    
if 'ora_store' in dir() and ora_store:
    try:
        ora_store.connection.close()
    except:
        pass

🧹 Cleanup
✅ Dropped test table NOTEBOOK_TEST_COLLECTION
✅ Database connection closed


---
## Summary

This notebook demonstrated how **Agentic RAG** leverages **langchain-oracledb** for enterprise-grade RAG:

### langchain-oracledb Components
- **OracleEmbeddings**: In-database embedding generation with ONNX models
- **OracleVS**: Vector storage and similarity search
- **OracleTextSplitter**: Server-side text chunking with normalization

### Multi-Collection Architecture
- PDF, Web, Repository, and General Knowledge collections
- Unified query interface across collections
- Metadata management and sanitization

### Multi-Agent CoT System
- Four specialized agents (Planner, Researcher, Reasoner, Synthesizer)
- A2A protocol for distributed communication
- Configurable deployment across multiple servers

### Next Steps
1. Start the API: `python -m src.main`
2. Launch Gradio UI: `python gradio_app.py`
3. Use the CLI: `python agent_cli.py`
4. Test A2A: `curl -X POST http://localhost:8000/a2a -d '{...}'`